# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-20-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-20-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-21 04:38:46,34.223334,-82.461707,242,2,0,240.0,"Abbeville, South Carolina, US",986.667754,0.826446
1,22001.0,Acadia,Louisiana,US,2020-07-21 04:38:46,30.295065,-92.414197,1844,53,0,1791.0,"Acadia, Louisiana, US",2972.036425,2.874187
2,51001.0,Accomack,Virginia,US,2020-07-21 04:38:46,37.767072,-75.632346,1053,15,0,1038.0,"Accomack, Virginia, US",3258.447828,1.424501
3,16001.0,Ada,Idaho,US,2020-07-21 04:38:46,43.452658,-116.241552,5979,33,0,5946.0,"Ada, Idaho, US",1241.520224,0.551932
4,19001.0,Adair,Iowa,US,2020-07-21 04:38:46,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34366,34451,34455,34740,34994,35070,35229,35301,35475,35526
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3371,3454,3571,3667,3752,3851,3906,4008,4090,4171
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,18712,19195,19689,20216,20770,21355,21948,22549,23084,23691


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,994,1010,1012,1048,1094,1113,1147,1164,1181,1185
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,89,93,95,97,101,104,107,111,112,113
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1004,1011,1018,1028,1040,1052,1057,1068,1078,1087


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,21135,21216,21254,21454,22456,22824,23151,23273,23634,23741
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1881,1946,2014,2062,2091,2137,2214,2264,2311,2352
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,13124,13743,14019,14295,14792,15107,15430,15744,16051,16400


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,310,310,312,312,313,314,314,314,314,319
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,33,33,33,36,36,37,37,37,37,38


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3080,1133.0,Winston,Alabama,US,2020-07-21 04:38:46,34.150305,-87.373259,350,8,0,342.0,"Winston, Alabama, US",1481.230691,2.285714
881,1055.0,Etowah,Alabama,US,2020-07-21 04:38:46,34.045673,-86.040519,1356,15,0,1341.0,"Etowah, Alabama, US",1325.927954,1.106195
2546,1117.0,Shelby,Alabama,US,2020-07-21 04:38:46,33.268798,-86.662326,2287,27,0,2260.0,"Shelby, Alabama, US",1050.518599,1.180586


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,3830010,140906,1160087
Brazil,2118646,80120,1514300
India,1155338,28082,724578


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,3830010,140906,1160087,2529017,2020-07-21 04:38:46,37.868262,-91.224568
Brazil,2118646,80120,1514300,524226,2020-07-21 04:38:46,-12.669522,-48.480493
India,1155338,28082,724578,402678,2020-07-21 04:38:46,23.356511,82.081064
Russia,776212,12408,552644,211160,2020-07-21 04:38:46,54.546312,62.120860
South Africa,373628,5173,194865,173590,2020-07-21 04:38:46,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,407326,32506,0
California,397870,7770,0
Florida,360394,5072,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,407326,32506,0,374820,2020-07-21 04:38:46,42.671593,-75.579694
California,397870,7770,0,390100,2020-07-21 04:38:46,37.778248,-120.728666
Florida,360394,5072,0,355322,2020-07-21 04:38:46,28.940755,-82.700744
Texas,343783,4106,0,339677,2020-07-21 04:38:46,31.647790,-98.615963
New Jersey,176963,15715,0,161248,2020-07-21 04:38:46,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,221703,23411,0
California,Los Angeles,159045,4104,0
Illinois,Cook,99052,4777,0
Arizona,Maricopa,96711,1485,0
Florida,Miami-Dade,87035,1309,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,221703,23411,0,198292,2020-07-21 04:38:46,40.767273,-73.971526,36061.0
California,Los Angeles,159045,4104,0,154941,2020-07-21 04:38:46,34.308284,-118.228241,6037.0
Illinois,Cook,99052,4777,0,94275,2020-07-21 04:38:46,41.841448,-87.816588,17031.0
Arizona,Maricopa,96711,1485,0,95226,2020-07-21 04:38:46,33.348359,-112.491815,4013.0
Florida,Miami-Dade,87035,1309,0,85726,2020-07-21 04:38:46,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,35301,4008,22549,880,687,76,122524,34462,11802,19573,...,295632,1044,16186,11483,382,8204,10,1581,2980,1478
2020-07-19,35475,4090,23084,880,705,76,126755,34877,12069,19655,...,296358,1054,16752,11891,383,8549,10,1606,2980,1611
2020-07-20,35526,4171,23691,884,749,76,130774,34981,12428,19743,...,296944,1064,17149,12334,384,8916,10,1619,3326,1713


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,1164,111,1068,52,29,3,2220,631,122,711,...,45358,33,83,110,0,59,1,443,120,25
2020-07-19,1181,112,1078,52,29,3,2260,641,123,711,...,45385,33,87,112,0,62,1,445,120,25
2020-07-20,1185,113,1087,52,29,3,2373,650,126,711,...,45397,33,90,116,0,63,1,447,128,26


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,23273,2264,15744,803,210,57,52607,23123,8274,17501,...,1413,921,9127,3972,357,1596,8,701,1462,439
2020-07-19,23634,2311,16051,803,221,57,54105,23294,8395,17599,...,1413,922,9279,3972,357,1921,8,712,1462,472
2020-07-20,23741,2352,16400,803,221,57,55913,23502,8398,17659,...,1413,927,9387,6844,360,1921,8,714,1620,472


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,4753,4904,5061,5206,5349,5514,5713,5899,6017,6150
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,951,1011,1069,1104,1136,1175,1236,1264,1307,1356
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,82,85,90,90,92,104,111,117,119,121
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,932,938,950,960,970,979,996,1011,1020,1027
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,96,98,101,106,110,113,118,134,143,146


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,65234,1796,0,141265,32533,380745,39770,47893,13429,49,...,76336,327106,33332,1338,283,76373,46026,4894,41485,2108
2020-07-19,67011,1875,0,143624,33228,386906,40124,47893,13519,49,...,78115,335257,34117,1350,283,77430,46946,4990,42315,2126
2020-07-20,68891,1950,0,145183,33927,397870,40548,48055,13624,49,...,79754,343783,34526,1360,304,78375,47743,5084,43018,2187


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-18         810    1796     465  260    421     392    671     724   
2020-07-19         823    1914     472  265    441     393    679     768   
2020-07-20         838    1990     479  274    465     395    682     814   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-18          730      158  ...      89      4       39       17   
2020-07-19          734      166  ...      89      4       39       17   
2020-07-20          739      172  ...      89      4       40       17   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-18            171   197   221          0       43      4  
2020-07-19            173   208   221          0       44      4  
2020-07-20            181   223   222          0       44      4  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,1286,18,0,2730,357,7702,1752,4396,523,0,...,838,3939,243,56,6,2025,1444,100,843,24
2020-07-19,1287,18,0,2761,357,7715,1752,4396,523,0,...,843,4012,243,56,6,2027,1447,100,844,25
2020-07-20,1291,18,0,2784,363,7770,1758,4406,523,0,...,847,4106,247,56,6,2031,1453,100,846,24


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-18          20      14       3    2      1      10     32       6   
2020-07-19          20      14       3    2      1      10     32       6   
2020-07-20          20      14       4    2      1      10     32       6   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-18           33        7  ...       0      0        0        0   
2020-07-19           33        7  ...       0      0        0        0   
2020-07-20           33        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-18              0     0     0         23        0      0  
2020-07-19              0     0     0         24        0      0  
2020-07-20              0     0     0         23        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.002044,0.026114,0.027383,0.000000,0.076803,0.0,0.027016,0.013558,0.031553,0.006893,...,0.002812,0.006750,0.037099,0.026092,0.000000,0.056672,0.0,0.003173,0.060498,0.040845
2020-07-19,0.004929,0.020459,0.023726,0.000000,0.026201,0.0,0.034532,0.012042,0.022623,0.004189,...,0.002456,0.009579,0.034968,0.035531,0.002618,0.042053,0.0,0.015813,0.000000,0.089986
2020-07-20,0.001438,0.019804,0.026295,0.004545,0.062411,0.0,0.031707,0.002982,0.029746,0.004477,...,0.001977,0.009488,0.023699,0.037255,0.002611,0.042929,0.0,0.008095,0.116107,0.063315


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.014821,0.037383,0.010407,0.0,0.0,0.0,0.019284,0.017742,0.033898,0.0,...,0.000883,0.03125,0.050633,0.028037,NaN,0.113208,0.0,0.006818,0.100917,0.041667
2020-07-19,0.014605,0.009009,0.009363,0.0,0.0,0.0,0.018018,0.015848,0.008197,0.0,...,0.000595,0.00000,0.048193,0.018182,NaN,0.050847,0.0,0.004515,0.000000,0.000000
2020-07-20,0.003387,0.008929,0.008349,0.0,0.0,0.0,0.050000,0.014041,0.024390,0.0,...,0.000264,0.00000,0.034483,0.035714,NaN,0.016129,0.0,0.004494,0.066667,0.040000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.005270,0.022584,0.020350,0.0,0.055276,0.0,0.056790,0.028054,0.013722,0.009576,...,0.007128,0.004362,0.013773,0.031153,0.002809,0.069705,0.0,0.008633,0.008276,0.002283
2020-07-19,0.015512,0.020760,0.019499,0.0,0.052381,0.0,0.028475,0.007395,0.014624,0.005600,...,0.000000,0.001086,0.016654,0.000000,0.000000,0.203634,0.0,0.015692,0.000000,0.075171
2020-07-20,0.004527,0.017741,0.021743,0.0,0.000000,0.0,0.033417,0.008929,0.000357,0.003409,...,0.000000,0.005423,0.011639,0.723061,0.008403,0.000000,0.0,0.002809,0.108071,0.000000


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,0.033967,0.035755,NaN,0.019795,0.024274,0.020056,0.011290,0.000000,0.006898,0.0,...,0.034097,0.029386,0.023333,0.002999,0.076046,0.012461,0.021279,0.023207,0.024144,0.018850
2020-07-19,0.027240,0.043987,NaN,0.016699,0.021363,0.016181,0.008901,0.000000,0.006702,0.0,...,0.023305,0.024919,0.023551,0.008969,0.000000,0.013840,0.019989,0.019616,0.020007,0.008539
2020-07-20,0.028055,0.040000,NaN,0.010855,0.021036,0.028338,0.010567,0.003383,0.007767,0.0,...,0.020982,0.025431,0.011988,0.007407,0.074205,0.012205,0.016977,0.018838,0.016613,0.028692


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-18      0.047865  0.076093  0.037946  0.035857  0.036946  0.010309   
2020-07-19      0.016049  0.065702  0.015054  0.019231  0.047506  0.002551   
2020-07-20      0.018226  0.039707  0.014831  0.033962  0.054422  0.005089   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2020-07-18      0.009023  0.029872  0.018131  0.053333  ...     0.0    0.0   
2020-07-19      0.011923  0.060773  0.005479  0.050633  ...     0.0    0.0   
2020-07-20      0.004418  0.059896  0.006812  0.036145  ...     0.0    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-18      0.000000      0.0   0.055556  0.053476  0.009132        NaN   
2020-07-19      0.000000      0.0   0.011696  0.055838  0.000000        NaN   
2020-07-20      0.025641      0.0   0.046243  0.072115  0.004525        NaN   

Province_State                      
Admin2          Washakie    Weston  
2020-07-18      0.000000  0.333333  
2020-07-19      0.023256  0.000000  
2020-07-20      0.000000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,0.016601,0.058824,NaN,0.056911,0.011331,0.013021,0.000571,0.000000,0.003839,NaN,...,0.028221,0.031152,0.034043,0.0,0.0,0.005961,0.006974,0.0,0.012005,0.000000
2020-07-19,0.000778,0.000000,NaN,0.011355,0.000000,0.001688,0.000000,0.000000,0.000000,NaN,...,0.005967,0.018533,0.000000,0.0,0.0,0.000988,0.002078,0.0,0.001186,0.041667
2020-07-20,0.003108,0.000000,NaN,0.008330,0.016807,0.007129,0.003425,0.002275,0.000000,NaN,...,0.004745,0.023430,0.016461,0.0,0.0,0.001973,0.004147,0.0,0.002370,-0.040000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin   Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-18         0.0  0.076923  0.000000  0.0    0.0     0.0    0.0     0.0   
2020-07-19         0.0  0.000000  0.000000  0.0    0.0     0.0    0.0     0.0   
2020-07-20         0.0  0.000000  0.333333  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-18      0.03125      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-19      0.00000      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-20      0.00000      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-18            NaN   NaN   NaN   0.000000      NaN    NaN  
2020-07-19            NaN   NaN   NaN   0.043478      NaN    NaN  
2020-07-20            NaN   NaN   NaN  -0.041667      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.003215,0.023151,0.027509,0.003267,0.064620,0.006829,0.032010,0.013854,0.028701,0.007078,...,0.002578,0.009236,0.036346,0.030495,0.002091,0.051820,8.278423e-10,0.009251,0.151001,0.065960
2020-07-19,0.004072,0.021805,0.025618,0.001634,0.045410,0.003414,0.033271,0.012948,0.025662,0.005634,...,0.002517,0.009407,0.035657,0.033013,0.002354,0.046936,4.139211e-10,0.012532,0.075500,0.077973
2020-07-20,0.002755,0.020805,0.025956,0.003089,0.053911,0.001707,0.032489,0.007965,0.027704,0.005056,...,0.002247,0.009447,0.029678,0.035134,0.002483,0.044933,2.069606e-10,0.010313,0.095804,0.070644


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.021396,0.033796,0.009099,2.282653e-12,0.017147,2.544076e-29,0.025738,0.019504,0.026628,0.000331,...,0.001495,0.020057,0.052898,0.031195,NaN,0.093722,0.0,0.008385,0.449367,0.055356
2020-07-19,0.018001,0.021403,0.009231,1.141327e-12,0.008573,1.272038e-29,0.021878,0.017676,0.017413,0.000165,...,0.001045,0.010028,0.050545,0.024688,NaN,0.072285,0.0,0.006450,0.224684,0.027678
2020-07-20,0.010694,0.015166,0.008790,5.706633e-13,0.004287,6.360191e-30,0.035939,0.015858,0.020901,0.000083,...,0.000655,0.005014,0.042514,0.030201,NaN,0.044207,0.0,0.005472,0.145675,0.033839


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-18,0.011618,0.025595,0.021774,3.665927e-06,0.182082,0.001444,0.041239,0.027617,0.011005,0.007225,...,0.005234,0.003961,0.019325,0.079005,0.002759,0.103722,4.850638e-12,0.006534,0.011237,0.029778
2020-07-19,0.013565,0.023177,0.020637,1.832964e-06,0.117231,0.000722,0.034857,0.017506,0.012814,0.006412,...,0.002617,0.002524,0.017990,0.039503,0.001379,0.153678,2.425319e-12,0.011113,0.005619,0.052474
2020-07-20,0.009046,0.020459,0.021190,9.164818e-07,0.058616,0.000361,0.034137,0.013218,0.006586,0.004911,...,0.001308,0.003973,0.014814,0.381282,0.004891,0.076839,1.212660e-12,0.006961,0.056845,0.026237


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,0.033449,0.033051,NaN,0.023511,0.023395,0.022906,0.012972,0.001281,0.009111,1.600673e-38,...,0.033534,0.035503,0.023202,0.004760,0.069901,0.012825,0.020852,0.024671,0.023270,0.019814
2020-07-19,0.030345,0.038519,NaN,0.020105,0.022379,0.019544,0.010936,0.000641,0.007906,8.003366e-39,...,0.028419,0.030211,0.023377,0.006864,0.034950,0.013333,0.020420,0.022143,0.021639,0.014176
2020-07-20,0.029200,0.039259,NaN,0.015480,0.021708,0.023941,0.010752,0.002012,0.007837,4.001683e-39,...,0.024701,0.027821,0.017682,0.007136,0.054578,0.012769,0.018699,0.020491,0.019126,0.021434


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-18      0.032668  0.066451  0.029801  0.031641  0.041900  0.009139   
2020-07-19      0.024359  0.066076  0.022428  0.025436  0.044703  0.005845   
2020-07-20      0.021292  0.052892  0.018629  0.029699  0.049562  0.005467   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-18      0.006687  0.045361  0.014725  0.043478  ...  0.004594   
2020-07-19      0.009305  0.053067  0.010102  0.047055  ...  0.002297   
2020-07-20      0.006861  0.056482  0.008457  0.041600  ...  0.001149   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-07-18      2.232691e-06  0.021505  0.156211   0.038358  0.046546   
2020-07-19      1.116345e-06  0.010753  0.078106   0.025027  0.051192   
2020-07-20      5.581727e-07  0.018197  0.039053   0.035635  0.061654   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-07-18      0.013087        NaN  0.000090  0.197857  
2020-07-19      0.006544        NaN  0.011673  0.098928  
2020-07-20      0.005534        NaN  0.005837  0.049464  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-18,0.021231,0.029430,NaN,0.045031,0.018508,0.014253,0.001676,0.000862,0.002348,NaN,...,0.024583,0.036287,0.022353,8.466573e-12,1.101160e-21,0.005374,0.005801,0.004651,0.007556,0.012864
2020-07-19,0.011004,0.014715,NaN,0.028193,0.009254,0.007970,0.000838,0.000431,0.001174,NaN,...,0.015275,0.027410,0.011177,4.233286e-12,5.505801e-22,0.003181,0.003939,0.002325,0.004371,0.027265
2020-07-20,0.007056,0.007358,NaN,0.018262,0.013030,0.007550,0.002131,0.001353,0.000587,NaN,...,0.010010,0.025420,0.013819,2.116643e-12,2.752901e-22,0.002577,0.004043,0.001163,0.003370,-0.006367


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                     \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount       Bullock   
2020-07-18      0.028736  0.054864  0.015640  0.031250    0.0  3.298902e-06   
2020-07-19      0.014368  0.027432  0.007820  0.015625    0.0  1.649451e-06   
2020-07-20      0.007184  0.013716  0.170577  0.007812    0.0  8.247256e-07   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-07-18      0.010290  0.006250  0.018576  2.671648e-09  ...     NaN   
2020-07-19      0.005145  0.003125  0.009288  1.335824e-09  ...     NaN   
2020-07-20      0.002573  0.001563  0.004644  6.679121e-10  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-07-18        NaN      NaN      NaN        NaN   NaN   NaN   0.013480   
2020-07-19        NaN      NaN      NaN        NaN   NaN   NaN   0.028479   
2020-07-20        NaN      NaN      NaN        NaN   NaN   NaN  -0.006594   

Province_State                  
Admin2         Washakie Weston  
2020-07-18          NaN    NaN  
2020-07-19          NaN    NaN  
2020-07-20          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200720,AK,2414,173172,NaN,29.0,NaN,NaN,NaN,0.0,...,175586,0,0,982ac1a1537b9b7a5d2dd768e419cece11dfe669,0,0,0,0,0,NaN
1,20200720,AL,68891,524897,NaN,1563.0,8368.0,NaN,994.0,NaN,...,593788,4,586,78b0f0cdb400a84b5052f80e597bc11138f5848d,0,0,0,0,0,NaN
2,20200720,AR,33927,398889,NaN,471.0,2202.0,NaN,NaN,111.0,...,432816,0,25,3bb85a05440526c23d60b70ad3388d236332589c,0,0,0,0,0,NaN
3,20200720,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,eb1949a094a8c4c95f33a9c6db8d407e917f5bdb,0,0,0,0,0,NaN
4,20200720,AZ,145183,648340,NaN,3084.0,6698.0,886.0,NaN,622.0,...,793523,23,66,181888e74b302a4e4e3e90a0771e139657f9f0fe,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200720,AK,2414.0,173172.0,NaN,29.0,NaN,NaN,NaN,0.0,...,175586,0,0,982ac1a1537b9b7a5d2dd768e419cece11dfe669,0,0,0,0,0,NaN
1,20200720,AL,68891.0,524897.0,NaN,1563.0,8368.0,NaN,994.0,NaN,...,593788,4,586,78b0f0cdb400a84b5052f80e597bc11138f5848d,0,0,0,0,0,NaN
2,20200720,AR,33927.0,398889.0,NaN,471.0,2202.0,NaN,NaN,111.0,...,432816,0,25,3bb85a05440526c23d60b70ad3388d236332589c,0,0,0,0,0,NaN
3,20200720,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,eb1949a094a8c4c95f33a9c6db8d407e917f5bdb,0,0,0,0,0,NaN
4,20200720,AZ,145183.0,648340.0,NaN,3084.0,6698.0,886.0,NaN,622.0,...,793523,23,66,181888e74b302a4e4e3e90a0771e139657f9f0fe,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-20,AK,2414,173172,NaN,29.0,NaN,NaN,NaN,0.0,NaN,...,175586,0,0,982ac1a1537b9b7a5d2dd768e419cece11dfe669,0,0,0,0,0,NaN
2020-07-20,AL,68891,524897,NaN,1563.0,8368.0,NaN,994.0,NaN,528.0,...,593788,4,586,78b0f0cdb400a84b5052f80e597bc11138f5848d,0,0,0,0,0,NaN
2020-07-20,AR,33927,398889,NaN,471.0,2202.0,NaN,NaN,111.0,309.0,...,432816,0,25,3bb85a05440526c23d60b70ad3388d236332589c,0,0,0,0,0,NaN
2020-07-20,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,eb1949a094a8c4c95f33a9c6db8d407e917f5bdb,0,0,0,0,0,NaN
2020-07-20,AZ,145183,648340,NaN,3084.0,6698.0,886.0,NaN,622.0,NaN,...,793523,23,66,181888e74b302a4e4e3e90a0771e139657f9f0fe,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-20,AK,2414.0,173172.0,NaN,29.0,NaN,NaN,NaN,0.0,NaN,...,175586,0,0,982ac1a1537b9b7a5d2dd768e419cece11dfe669,0,0,0,0,0,NaN
2020-07-20,AL,68891.0,524897.0,NaN,1563.0,8368.0,NaN,994.0,NaN,528.0,...,593788,4,586,78b0f0cdb400a84b5052f80e597bc11138f5848d,0,0,0,0,0,NaN
2020-07-20,AR,33927.0,398889.0,NaN,471.0,2202.0,NaN,NaN,111.0,309.0,...,432816,0,25,3bb85a05440526c23d60b70ad3388d236332589c,0,0,0,0,0,NaN
2020-07-20,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,eb1949a094a8c4c95f33a9c6db8d407e917f5bdb,0,0,0,0,0,NaN
2020-07-20,AZ,145183.0,648340.0,NaN,3084.0,6698.0,886.0,NaN,622.0,NaN,...,793523,23,66,181888e74b302a4e4e3e90a0771e139657f9f0fe,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE